In [73]:
!pip install openai

In [91]:
# Required packages to install
# pip install openai requests pydantic

# Required imports
import json
import requests
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from openai import OpenAI  # For chat completions API
import asyncio

# Initialize the OpenAI client
client = OpenAI(api_key="your-openai-api-key")  

# For the vision model, you need either:
# Option 1: If using OpenAI's Vision model
# (already included in the OpenAI package)

# Option 2: If using another vision model like Google's Vertex AI
# pip install google-cloud-aiplatform
# from vertexai.vision_models import ImageCaptioningModel
# vision_model = ImageCaptioningModel.from_pretrained("your-model-name")

In [75]:
# Define the schema for product search parameters
class AmazonSearchQuery(BaseModel):
    product_name: str = Field(..., description="Main product name extracted from the image")
    brand: Optional[str] = Field(None, description="Brand name if visible in the image")
    product_size: Optional[str] = Field(None, description="Size/amount/weight information (e.g., '16oz', '500ml')")
    identifiers: Optional[Dict[str, str]] = Field(None, description="Product identifiers such as UPC, EAN, ASIN if visible")
    key_features: Optional[List[str]] = Field(None, description="Key product features or descriptors visible in the image")
    
# New schema for multiple products in an image
class ImageProductList(BaseModel):
    products: List[AmazonSearchQuery] = Field(..., description="List of products identified in the image")
   
# Define the schema for Amazon search results verification
class ProductVerification(BaseModel):
    match_found: bool = Field(..., description="Whether a matching product was found on Amazon")
    confidence_score: float = Field(..., description="Confidence score for the match (0-1)")
    matched_product: Optional[Dict] = Field(None, description="Details of the matched product")
    discrepancies: Optional[List[str]] = Field(None, description="List of discrepancies between image and found product")
    amazon_link: Optional[str] = Field(None, description="Link to the Amazon product page if found")

In [76]:
async def amazon_search_async(search_parameters):
    """
    Asynchronous version of Amazon search for products matching parameters extracted from a product image.
    
    Input: Structured data from VLM image analysis
    Output: Top matching Amazon products with detailed information
    """
    import aiohttp
    
    api_key = "FB0C32E1DCB3433C997C0A0FB1E70608"
    base_url = "https://api.rainforestapi.com/request"
    
    # Configure request parameters
    params = {
        "api_key": api_key,
        "amazon_domain": "amazon.com"
    }
    
    # Determine search approach based on available information
    if search_parameters.get("identifiers") and search_parameters["identifiers"].get("asin"):
        # Direct ASIN lookup if available
        params["type"] = "product"
        params["asin"] = search_parameters["identifiers"]["asin"]
    elif search_parameters.get("identifiers") and any(k in search_parameters["identifiers"] for k in ["upc", "ean", "isbn"]):
        # GTIN/UPC/EAN lookup
        params["type"] = "product"
        identifiers = search_parameters["identifiers"]
        params["gtin"] = identifiers.get("upc") or identifiers.get("ean") or identifiers.get("isbn")
    else:
        # Text-based search using extracted information
        params["type"] = "search"
        
        # Construct search term from product name and attributes
        search_term = search_parameters["product_name"]
        if search_parameters.get("brand"):
            search_term = f"{search_parameters['brand']} {search_term}"
        if search_parameters.get("product_size"):
            search_term += f" {search_parameters['product_size']}"
        if search_parameters.get("key_features"):
            search_term += f" {' '.join(search_parameters['key_features'][:3])}"  # Add top 3 features
        
        params["search_term"] = search_term
    
    # Make API request
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(base_url, params=params) as response:
                if response.status != 200:
                    return {
                        "query_info": search_parameters,
                        "error": f"API request failed with status {response.status}",
                        "results": []
                    }
                
                data = await response.json()
                
                # Process results based on request type
                results = []
                if params["type"] == "product":
                    if "product" in data:
                        product = data["product"]
                        results.append({
                            "asin": product.get("asin"),
                            "title": product.get("title"),
                            "brand": product.get("brand"),
                            "price": product.get("buybox_winner", {}).get("price"),
                            "rating": product.get("rating"),
                            "ratings_total": product.get("ratings_total"),
                            "main_image": product.get("main_image", {}).get("link"),
                            "link": product.get("link"),
                            "dimensions": product.get("dimensions"),
                            "weight": product.get("weight")
                        })
                elif params["type"] == "search":
                    for item in data.get("search_results", [])[:5]:  # Limit to top 5 results
                        results.append({
                            "asin": item.get("asin"),
                            "title": item.get("title"),
                            "link": item.get("link"),
                            "image": item.get("image"),
                            "price": item.get("price")
                        })
                
                return {
                    "query_info": search_parameters,
                    "results": results
                }
    except Exception as e:
        # Handle any errors
        return {
            "query_info": search_parameters,
            "error": str(e),
            "results": []
        }

In [77]:
def amazon_search(search_parameters):
    """
    Search Amazon for products matching parameters extracted from a product image.
    
    Input: Structured data from VLM image analysis
    Output: Top matching Amazon products with detailed information
    """
    import requests
    
    api_key = "FB0C32E1DCB3433C997C0A0FB1E70608"
    base_url = "https://api.rainforestapi.com/request"
    
    # Configure request parameters
    params = {
        "api_key": api_key,
        "amazon_domain": "amazon.com"
    }
    
    # Determine search approach based on available information
    if "asin" in search_parameters and search_parameters["asin"]:
        # Direct ASIN lookup if available
        params["type"] = "product"
        params["asin"] = search_parameters["asin"]
    elif "upc" in search_parameters or "ean" in search_parameters or "isbn" in search_parameters:
        # GTIN/UPC/EAN lookup
        params["type"] = "product"
        params["gtin"] = search_parameters.get("upc") or search_parameters.get("ean") or search_parameters.get("isbn")
    else:
        # Text-based search using extracted information
        params["type"] = "search"
        
        # Construct search term from product name and attributes
        search_term = search_parameters["product_name"]
        if "brand" in search_parameters:
            search_term = f"{search_parameters['brand']} {search_term}"
        if "attributes" in search_parameters:
            for key, value in search_parameters["attributes"].items():
                search_term += f" {value}"
        
        params["search_term"] = search_term
    
    # Make API request
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise exception for HTTP errors
        data = response.json()
        
        # Process results based on request type
        results = []
        if params["type"] == "product":
            if "product" in data:
                product = data["product"]
                results.append({
                    "asin": product.get("asin"),
                    "title": product.get("title"),
                    "brand": product.get("brand"),
                    "price": product.get("buybox_winner", {}).get("price"),
                    "rating": product.get("rating"),
                    "ratings_total": product.get("ratings_total"),
                    "main_image": product.get("main_image", {}).get("link"),
                    "link": product.get("link"),
                    "dimensions": product.get("dimensions"),
                    "weight": product.get("weight")
                })
        elif params["type"] == "search":
            for item in data.get("search_results", [])[:5]:  # Limit to top 5 results
                results.append({
                    "asin": item.get("asin"),
                    "title": item.get("title"),
                    "link": item.get("link"),
                    "image": item.get("image"),
                    "price": item.get("price")
                })
        
        return {
            "query_info": search_parameters,
            "results": results
        }
    except Exception as e:
        # Handle any errors
        return {
            "query_info": search_parameters,
            "error": str(e),
            "results": []
        }

In [78]:
def extract_product_details_from_image_mixtral(image_path, client, model_id="accounts/fireworks/models/mixtral-8x7b-instruct", timing=None):
    """
    Extract details for multiple products from an image using document inlining and a specified model.
    
    Parameters:
    image_path (str): Path to the product image file
    client: API client (OpenAI or compatible)
    model_id (str): Model ID to use for extraction
    timing (dict, optional): Timing dictionary to update with performance metrics
    
    Returns:
    dict: List of extracted product details
    """
    import base64
    import time
    import json
    
    # Track timing if requested
    if timing is not None:
        image_prep_start = time.time()
    
    # Prepare image for document inlining
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()
        
    base64_image = base64.b64encode(image_content).decode('utf-8')
    
    # Determine MIME type based on file extension
    if image_path.lower().endswith('.jpg') or image_path.lower().endswith('.jpeg'):
        mime_type = "image/jpeg"
    elif image_path.lower().endswith('.png'):
        mime_type = "image/png"
    else:
        mime_type = "image/jpeg"  # Default to JPEG
    
    # Create image URL with document inlining transform
    image_url = f"data:{mime_type};base64,{base64_image}#transform=inline"
    
    if timing is not None:
        timing['image_preparation'] = time.time() - image_prep_start
        extract_start = time.time()
    
    # Extract multiple product details from image using document inlining
    extract_messages = [
        {"role": "system", "content": "Extract detailed information about all products visible in the image. For each product, include product name, brand, size/weight, and any visible identifiers like UPC or model numbers."},
        {"role": "user", "content": [
            {"type": "image_url", "image_url": {"url": image_url}},
            {"type": "text", "text": "What products are shown in this image? Identify each distinct product and extract all visible details including product name, brand, size/weight, and any identifiers for each. Return as a list of products."}
        ]}
    ]
    
    # Use document inlining to extract product details with JSON output
    extract_response = client.chat.completions.create(
        model=model_id,
        messages=extract_messages,
        response_format={"type": "json_object", "schema": ImageProductList.model_json_schema()},
        max_tokens=1500
    )
    
    # Parse extracted product details
    product_details = json.loads(extract_response.choices[0].message.content)
    
    if timing is not None:
        timing['product_extraction'] = time.time() - extract_start
    
    return product_details

In [79]:
def extract_product_details_from_image_firesearch(image_path, client, timing=None):
    """
    Extract product details from an image using the firesearch-ocr-v6 model.
    
    Parameters:
    image_path (str): Path to the product image file
    client: API client (OpenAI compatible)
    timing (dict, optional): Timing dictionary to update with performance metrics
    
    Returns:
    dict: Extracted product details
    """
    import base64
    import time
    import json
    
    # Track timing if requested
    if timing is not None:
        image_prep_start = time.time()
    
    # Encode image to base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Determine MIME type based on file extension
    if image_path.lower().endswith('.jpg') or image_path.lower().endswith('.jpeg'):
        mime_type = "image/jpeg"
    elif image_path.lower().endswith('.png'):
        mime_type = "image/png"
    else:
        mime_type = "image/jpeg"  # Default to JPEG
        
    if timing is not None:
        timing['image_preparation'] = time.time() - image_prep_start
        extract_start = time.time()
    
    # Extract product details using firesearch-ocr-v6 model
    extract_messages = [
        {"role": "system", "content": "You are an OCR and product information extraction assistant. Extract detailed product information from the image. Include product name, brand, size/weight, and any visible identifiers like UPC, EAN, or model numbers."},
        {"role": "user", "content": [
            {"type": "text", "text": "Extract all product information visible in this image. Include the product name, brand, size/weight, and any identifiers like UPC, EAN, or model numbers. Format your response as JSON."},
            {"type": "image_url", "image_url": {"url": f"data:{mime_type};base64,{base64_image}"}}
        ]}
    ]
    
    # Make API call to firesearch-ocr model
    extract_response = client.chat.completions.create(
        model="accounts/fireworks/models/phi-3-vision-128k-instruct",
        messages=extract_messages,
        response_format={"type": "json_object", "schema": AmazonSearchQuery.model_json_schema()},
        max_tokens=1000
    )
    
    # Parse extracted product details
    product_details = json.loads(extract_response.choices[0].message.content)
    
    if timing is not None:
        timing['product_extraction'] = time.time() - extract_start
    
    return product_details

In [87]:
async def process_product_image_async(image_path, api_key):
    """
    Process a product image containing multiple products using document inlining and JSON mode.
    Makes asynchronous API calls to search for each product.
    
    Parameters:
    image_path (str): Path to the product image file
    api_key (str): Fireworks API key
    
    Returns:
    dict: Product verification results for all products in the image
    """
    import time
    import json
    import aiohttp
    
    # Track timing information
    timing = {}
    start_total = time.time()
    
    # Initialize client
    client_start = time.time()
    client = OpenAI(
        base_url="https://api.fireworks.ai/inference/v1",
        api_key=api_key
    )
    timing['client_initialization'] = time.time() - client_start
    
    # Extract product details from the image
    extract_start = time.time()
    product_details = extract_product_details_from_image_mixtral(
        image_path=image_path,
        client=client,
        timing=timing
    )
    timing['product_extraction'] = time.time() - extract_start
    
    # Process each product asynchronously
    search_start = time.time()
    
    # Create list of tasks for each product
    search_tasks = []
    products_list = product_details.get("products", [])
    
    for product in products_list:
        search_tasks.append(amazon_search_async(product))
        
    # Run all search tasks concurrently
    amazon_results = await asyncio.gather(*search_tasks)
    timing['amazon_search'] = time.time() - search_start
    
    # Create the final output structure - simplified for just top matches
    final_results = {
        "products_found": len(products_list),
        "product_matches": [],
        "timing": timing
    }
    
    # Process each product's top match with simplified output
    for i, (product, amazon_result) in enumerate(zip(products_list, amazon_results)):
        product_match = {
            "product_name": product.get("product_name"),
            "brand": product.get("brand")
        }
        
        # If we have at least one result, add the top match with only requested fields
        if amazon_result.get("results") and len(amazon_result["results"]) > 0:
            top_result = amazon_result["results"][0]
            
            # Extract just the price value if it's a complex object
            price = top_result.get("price", "")
            if isinstance(price, dict) and "raw" in price:
                price = price["raw"]
            
            # Create simplified top match with only requested fields
            product_match["top_match"] = {
                "title": top_result.get("title", ""),
                "asin": top_result.get("asin", ""),
                "price": price,
                "url": top_result.get("link", "")
            }
        else:
            product_match["top_match"] = None
        
        final_results["product_matches"].append(product_match)
    
    timing['total_time'] = time.time() - start_total
    
    # Print timing information
    print("\n=== Timing Information ===")
    for step, duration in timing.items():
        print(f"{step}: {duration:.2f} seconds")
    print("==========================\n")
    
    return final_results

In [93]:
def process_product_image(image_path, api_key):
    """
    Process a product image with multiple products, compatible with Jupyter notebooks.
    
    Parameters:
    image_path (str): Path to the product image file
    api_key (str): Fireworks API key
    
    Returns:
    dict: Simplified product verification results with only top matches
    """
    # Get the current event loop instead of creating a new one
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(process_product_image_async(image_path, api_key))

In [81]:
# results = process_product_image("tea.png", "fw_3ZYLXEz2c3YWcN3CAkyaVedA")
# print(json.dumps(results, indent=2))

In [95]:
results = process_product_image("tea.png", "fw_3ZYLXEz2c3YWcN3CAkyaVedA")
print(json.dumps(results, indent=2))

RuntimeError: This event loop is already running

## Amazon Search Test

In [83]:
# Sample input parameters (as if from VLM analyzing a product image)
search_parameters = {
    "product_name": "Pro-V Daily Moisture Renewal Shampoo",
    "brand": "Pantene",
    "attributes": {
        "size": "25.4 fl oz",
        "color": "white bottle with gold cap",
        "description": "For dry hair"
    },
}

In [84]:
# # Import necessary libraries
# import json
# import requests

# # Call the function with sample parameters
# results = amazon_search(search_parameters)

# # Print the results in a readable format
# print(json.dumps(results, indent=2))

In [85]:
import requests
import json

def simple_rainforest_test(api_key):
    """
    Simple test of the Rainforest API using a basic search query
    """
    # Base URL for Rainforest API
    base_url = "https://api.rainforestapi.com/request"
    
    # Simple search parameters
    params = {
        "api_key": api_key,
        "type": "search",
        "amazon_domain": "amazon.com",
        "search_term": "iPhone charger",  # Simple, common product
        "sort_by": "featured"
    }
    
    # Make the request
    response = requests.get(base_url, params=params)
    
    # Check if request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Return basic info about the results
        result_summary = {
            "request_info": data.get("request_info", {}),
            "total_results": len(data.get("search_results", [])),
            "first_result": data.get("search_results", [{}])[0] if data.get("search_results") else None
        }
        
        return result_summary
    else:
        return {
            "error": f"Request failed with status code {response.status_code}",
            "response": response.text
        }

# # Example usage:
# api_key = "FB0C32E1DCB3433C997C0A0FB1E70608"  # Replace with your actual API key
# results = simple_rainforest_test(api_key)
# print(json.dumps(results, indent=2))